### Import libraries

In [ ]:
# import random
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from ultralytics import YOLO

### Define functions

In [2]:
def detect_persons(image):
    model = YOLO("../weights/yolov9e-seg.pt")
    classes_ids = [0]  # person
    conf = 0.2

    output = []

    results = model.predict(image, conf=conf)
    result = results[0]
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        if int(box.cls[0]) in classes_ids:

            # Crop the image
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cropped_img = image[y1:y2, x1:x2]

            # Create a mask for the cropped image
            mask_img = np.zeros_like(cropped_img)
            mask_points = points - [x1, y1]
            cv2.fillPoly(mask_img, mask_points, (255, 255, 255))

            # Apply the mask to the cropped image
            masked_cropped_img = cv2.bitwise_and(cropped_img, mask_img)

            # add person to output
            output.append({
                "image": masked_cropped_img,
                "box": box,
                "mask": mask
            })

    return output

In [3]:
def detect_clothes(image):
    model = YOLO("../weights/clothes.pt")

    output = []
    
    results = model.predict(image)
    result = results[0]
    for box in result.boxes:
        output.append({
            "image": image,
            "box": box
        })

    return output

### Detection

In [4]:
# input (change the path to your image path)
image_path = "../dataset/clothes/test/mk.jpg"

In [ ]:
# detection process

image = cv2.imread(os.path.join(image_path))

# show the original image
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

classes = ["mask", "hat", "light_top", "heavy_top", "pants", "shorts_skirt"]
classes_id = [0]
colors = [(0, 255, 0), (0, 0, 255)]

mask_person_count = 0
final_image = image.copy()
persons = detect_persons(image)
print(f"{len(persons)} people detected")
for person in persons:
    clothes = detect_clothes(person["image"])
    # print(f"{len(clothes)} clothes detected on this person:", end=" ")

    # add the bounding box and semi-transparent color mask to final_image
    x1, y1, x2, y2 = map(int, person["box"].xyxy[0])
    color_number = 0
    cv2.rectangle(final_image, (x1, y1), (x2, y2), colors[color_number], 5)
    # cv2.putText(final_image, f"person {person['box'].conf[0]:.2f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[color_number], 2)

    person_image = person["image"]

    # # show original person image
    # plt.imshow(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    # # save the image
    # cv2.imwrite("person.jpg", person_image)

    for cloth in clothes:
        if cloth["box"].cls[0] in classes_id:
            print(f"{classes[int(cloth["box"].cls[0])]}: {cloth["box"].conf[0]:.2f}")
            # print(cloth["box"])
            x1, y1, x2, y2 = map(int, cloth["box"].xyxy[0])

            # add the bounding box to person_image
            color_number = 1
            cv2.rectangle(person_image, (x1, y1), (x2, y2), colors[color_number], 3)
            # add the class name
            # cv2.putText(person_image, f"{classes[int(cloth["box"].cls[0])]} {cloth["box"].conf[0]:.2f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[color_number], 2)

            # add the bounding box to final_image
            x1, y1, x2, y2 = map(int, cloth["box"].xyxy[0])
            x1 += int(person["box"].xyxy[0][0])
            y1 += int(person["box"].xyxy[0][1])
            x2 += int(person["box"].xyxy[0][0])
            y2 += int(person["box"].xyxy[0][1])
            # color_number = 1
            cv2.rectangle(final_image, (x1, y1), (x2, y2), colors[color_number], 3)
            # cv2.putText(final_image, f"{classes[int(cloth['box'].cls[0])]} {cloth['box'].conf[0]:.2f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[color_number], 2)

            # # Draw the mask
            # points = np.int32([cloth["mask"]])
            # cv2.polylines(person_image, points, True, (255, 0, 0), 1)
            # cv2.fillPoly(person_image, points, colors[color_number])

            mask_person_count += 1
            break # detect only one mask per person

    plt.imshow(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

print(f"{mask_person_count} people are wearing masks in {len(persons)} people. (At least {mask_person_count/len(persons)*100:.2f}%)")

# save the image
cv2.imwrite("output.jpg", final_image)
